# <center> MobileNetV2 x1.4 Adversarial Example Generation </center>

We are using HopSkipJump un-targeted attack algorithm to generate adversarial examples using the Adversarial - Robustness - Toolbox

### Libraries and Definitions.

We use 0 seed to maintain reproducibility.

In [ ]:
from torch.quantization import MovingAverageMinMaxObserver
from torch.ao.quantization.observer import MinMaxObserver
from torch.quantization import QuantStub, DeQuantStub
import torch.nn as nn
import urllib
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import torch.optim as optim
import numpy as np
from torchvision import datasets
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10
from art.preprocessing.standardisation_mean_std import StandardisationMeanStdPyTorch
from art.attacks.evasion.hop_skip_jump import HopSkipJump
import time
import pickle
import os
from math import log10, sqrt
import  random
import cv2
from torch.utils.data import DataLoader
torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    epsilon = 1e-10  # Small epsilon to avoid division by zero
    mse = max(mse, epsilon)  # Ensure MSE is not zero
    max_pixel = 1.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

def print_size_of_model(model):
    """ Prints the real size of the model """
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

    
def accuracy(output, target):
    """ Computes the top 1 accuracy """
    with torch.no_grad():
        batch_size = target.size(0)

        _, pred = output.topk(1, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        correct_one = correct[:1].view(-1).float().sum(0, keepdim=True)
        return correct_one.mul_(100.0 / batch_size).item()

def normalize_np(img):
  img = torch.from_numpy(img)
  img = F.normalize(img, [0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
  return img.numpy()


def test(model: nn.Module, dataloader: DataLoader, cuda=False) -> float:
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            if cuda:
              inputs = inputs.cuda()
              labels = labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

### Dataset Preparation

In [ ]:
# Step 1: Load the CIFAR10 dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_cifar10()

# Step 1a: Swap axes to PyTorch's NCHW format

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_data = datasets.CIFAR10(root="./data",train=False,download=True,transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

# show images
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


### Adversarial Examples Dataset Generation

The following code generates adversarial examples for each model "version".

1. 30000 examples
2. 20000 examples
3. 10000 examples

Due to the amount of images processed, we split each loop into 300s, 200s, 100s respectively.
In the end, each dataset is consisted of 100 files which we merge into one. 
The files are available on the drive link if you are interested.

Each of the following blocks loads the corresponding model and generates its examples for the dataset

In [ ]:
#----------------------------------------------------------------------     FIRST ADVERSARIAL EXAMPLE GENERATION ----------------------------------------------------------------------


model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_mobilenetv2_x1_4", pretrained=True)
model.to("cuda")
model.eval()
model.float()


criterion = nn.CrossEntropyLoss()

#Define the optimizer used for each model (Fully optional, can be omitted)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, dampening=0,weight_decay=0.0005, nesterov=True)


score = test(model, test_loader, cuda=True)
print('Initial accuracy on test images: {}% - FP32'.format(score))

# Create the ART - PyTorch classifier
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0.0, 1.0),
    loss=criterion,
    optimizer=optimizer,
    preprocessing=((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)), #Normalization step shouldn't happen at the data. It's inserted here.
    input_shape=(3, 32, 32),
    nb_classes=10,
    device_type = "gpu"
)
print("HopSkipJump Attack Initialization")

attack = HopSkipJump(classifier,64,targeted = False,verbose = True)

for i in range(0,100):
    x_train_hop_mobile_cifar = attack.generate(normalize_np(x_train[0+300*i:300*(i+1)]),y_train[0+300*i:300*(i+1)])
    with open("x_1_train_hop_mobile_cifar_{}_to_{}.pkl".format(300*i,300*(i+1)),'wb') as f:
        pickle.dump(x_train_hop_mobile_cifar,f)
    with open("mobilenet_1cifar_progress.txt",'wb') as f:
        f.write("Done with {}\n".format(300*(i+1)))
    print("{} Examples generated.".format(300*(i+1)))

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------     SECOND ADVERSARIAL TRAINING ITERATION ----------------------------------------------------------------------

################################################################################################################################################
"""REMINDER: Load the retrained model, not the initial."""


adversarial_state = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1umVJExm8VeVSqIpEWlCVhMbJrNZeU-aw&export=download&confirm=t&uuid=0")

"""
If you want to manually load the file or load your own model version, use the code below updating your path accordingly

# Adversarial Model 1 https://drive.google.com/file/d/1umVJExm8VeVSqIpEWlCVhMbJrNZeU-aw/view?usp=sharing
"""


#path = "~/Downloads/"

#adversarial_state = os.path.join(path, "MobileNet_1it_CIFAR10_94.17acc.pkl")

model.load_state_dict(torch.load(adversarial_state))
model.float()
model.eval()
################################################################################################################################################


#Define the optimizer used for each model (Fully optional, can be omitted)
optimizer = optim.Adam(model.parameters(), lr=0.0001)


score = test(model, test_loader, cuda=True)
print('Initial accuracy on test images: {}% - FP32'.format(score))

# Create the ART - PyTorch classifier
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0.0, 1.0),
    loss=criterion,
    optimizer=optimizer,
    preprocessing=((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)), #Normalization step shouldn't happen at the data. It's inserted here.
    input_shape=(3, 32, 32),
    nb_classes=10,
    device_type = "gpu"
)
print("HopSkipJump Attack Initialization")

attack = HopSkipJump(classifier,64,targeted = False,verbose = True)

for i in range(0,100):
    x_train_hop_mobile_cifar = attack.generate(normalize_np(x_train[0+300*i:300*(i+1)]),y_train[0+300*i:300*(i+1)])
    with open("x_2_train_hop_mobile_cifar_{}_to_{}.pkl".format(300*i,300*(i+1)),'wb') as f:
        pickle.dump(x_train_hop_mobile_cifar,f)
    with open("mobilenet_2cifar_progress.txt",'wb') as f:
        f.write("Done with {}\n".format(300*(i+1)))
    print("{} Examples generated.".format(300*(i+1)))

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------     THIRD ADVERSARIAL TRAINING ITERATION ----------------------------------------------------------------------

################################################################################################################################################
"""REMINDER: Load the retrained model, not the initial."""

adversarial_state = urllib.request.urlopen("https://drive.usercontent.google.com/download?id=12Ux4pWLxK4gTr54eMN-Mds4adPMvjfNX&export=download&confirm=t&uuid=0")

"""
If you want to manually load the file or load your own model version, use the code below updating your path accordingly

# Adversarial Model 2 https://drive.google.com/file/d/12Ux4pWLxK4gTr54eMN-Mds4adPMvjfNX/view?usp=sharing
"""


#path = "~/Downloads/"

#adversarial_state = os.path.join(path, "MobileNet_2it_CIFAR10_93.58acc.pkl")

model.load_state_dict(torch.load(adversarial_state))
model.float()
model.eval()
################################################################################################################################################

criterion = nn.CrossEntropyLoss()

#Define the optimizer used for each model (Fully optional, can be omitted)
optimizer = optim.Adam(model.parameters(), lr=5e-7)


score = test(model, test_loader, cuda=True)
print('Initial accuracy on test images: {}% - FP32'.format(score))

# Create the ART - PyTorch classifier
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0.0, 1.0),
    loss=criterion,
    optimizer=optimizer,
    preprocessing=((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)), #Normalization step shouldn't happen at the data. It's inserted here.
    input_shape=(3, 32, 32),
    nb_classes=10,
    device_type = "gpu"
)
print("HopSkipJump Attack Initialization")

attack = HopSkipJump(classifier,64,targeted = False,verbose = True)

for i in range(0,100):
    x_train_hop_mobile_cifar = attack.generate(normalize_np(x_train[0+300*i:300*(i+1)]),y_train[0+300*i:300*(i+1)])
    with open("x_3_train_hop_mobile_cifar_{}_to_{}.pkl".format(300*i,300*(i+1)),'wb') as f:
        pickle.dump(x_train_hop_mobile_cifar,f)
    with open("mobilenet_3cifar_progress.txt",'wb') as f:
        f.write("Done with {}\n".format(300*(i+1)))
    print("{} Examples generated.".format(300*(i+1)))
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------